In [1]:
import pandas as pd
import re
from functools import reduce

# STable 1: Cross ancestry meta analysis statistics

In [2]:
meta_file = "../data/meta/tables/all_ancestry.xlsx"
meta_df = pd.read_excel(meta_file, index_col=[0,1], header=[0,1,2])

# STable 2: Cross ancestry meta analysis using METAL

In [3]:
metal_file = "../data/metal/bmi_rint_monogenic_meta_metal.xlsx"
metal_df = pd.read_excel(metal_file, index_col=[0], header=[0,1,2])

# STable 3: Shadow effects

In [4]:
shadow_file = "../data/meta/tables/shadow_effect.xlsx"
shadow_df = pd.read_excel(shadow_file, index_col=[0,1], header=[0,1,2])

# STable 4: LOVO

In [5]:
lovo_file = "../data/meta/tables/lovo.xlsx"
lovo_df = pd.read_excel(lovo_file, index_col=[0,1,2], header=[0,1])

# STable 5: Ancestral heterogeneity

In [6]:
hetz_file = "../data/meta/tables/qstats_ancestry.tsv"
hetz_df = pd.read_csv(hetz_file, sep="\t")

# STable 6: Medications

In [7]:
med_file = "../data/meta/tables/meds.xlsx"
med_df = pd.read_excel(med_file, index_col=[0,1], header=[0,1,2])

# STable 7: Obesity clinical category enrichment

In [8]:
bmi_cat_file = "../data/enrichment/bmi_cat/monogenic_enrich_bmi_cat.xlsx"
bmi_cat_df = pd.read_excel(bmi_cat_file, index_col=[0,1], header=[0,1])

# STable 8: Comorbidity enrichment

In [9]:
comorbid_file = "../data/enrichment/comorbid/monogenic_enrich_comorbid.xlsx"
comorbid_df = pd.read_excel(comorbid_file, index_col=[0,1], header=[0,1])

# STable 9: Structural Equation Modelling

In [10]:
sem_file = "../data/sem/sem_meta.xlsx"
sem_df = pd.read_excel(sem_file, index_col=[0,1], header=[0])

# STable 10: Phewas

In [11]:
phewas_df = pd.DataFrame()
for genes in ["RIF1", "YLPM1", "GIGYF1", "SLC5A3", "GRM7"]:
    phewas_file = f"../data/phewas/{genes}/phewas_meta.xlsx"
    phewas_df = pd.concat([phewas_df, pd.read_excel(phewas_file, index_col=[0,1,2,3], header=[0,1])])

# STable 11: PGS interaction

In [12]:
pgs_file = "../data/pgs_interaction/meta_pgs_interaction.xlsx"
pgs_df = pd.read_excel(pgs_file, index_col=[0,1], header=[0])

# STable 12: BMI distribution by PGS quantiles

In [13]:
bmi_pgs_file = "../data/pgs_interaction/bmi_by_pgs_cat.xlsx"
bmi_pgs_df = pd.read_excel(bmi_pgs_file, index_col=[0], header=[0,1])

# STable 13: PTV protein association

In [14]:
ptv_protein_file = "../data/proteomics/assoc_protein_gene_carrier.csv.gz"
ptv_protein_df = pd.read_csv(ptv_protein_file, index_col=[-1, 0], header=[0])

# STable 14: Protein BMI assoc

In [15]:
protein_bmi_file = "../data/proteomics/protein_bmi_assoc.csv.gz"
protein_bmi_df = pd.read_csv(protein_bmi_file, index_col=[0,1], header=[0])

# STable 15: Cross ancestry previous genes

In [16]:
previous_meta_file = "../data/known_genes/monogenic_meta_known_genes.xlsx"
previous_meta_df = pd.read_excel(previous_meta_file, index_col=[0,1], header=[0,1,2])

# STable 16: Cross ancestry previous sex specific

In [17]:
previous_meta_sex_dfs = []

for sex in ["male", "female"]:
    pms_file = f"../data/known_genes/sex_specific_known_female.xlsx"
    pms_df = pd.read_excel(pms_file, index_col=[0,1], header=[0,1,2])
    pms_df = pd.concat([pms_df],  keys=[sex], names=["sex"], axis=1)
    previous_meta_sex_dfs.append(pms_df)

previous_meta_sex_df = pd.concat((previous_meta_sex_dfs), axis=1)
previous_meta_sex_df.columns = pd.MultiIndex.from_tuples(
            [(gene, mask, sex, stat) for sex, gene, mask, stat in previous_meta_sex_df.columns],
            names=["Gene", "Gene Mask", "Sex", "Statistic"]
        )

previous_meta_sex_df = previous_meta_sex_df.loc[:, 
    [(g,"pLoF",s,st) 
    for g in previous_meta_sex_df.columns.get_level_values("Gene").unique() 
    for s in ["female", "male"]
    for st in previous_meta_sex_df.columns.get_level_values("Statistic").unique()
    ]
]

# STable 17: Comorbidity diagnosis

In [18]:
comorbid_selection_df = pd.read_excel("../data/enrichment/comorbid/obesity_associated_diseases.xlsx", index_col=0)

# Organize dataframes

In [19]:
supp_dfs = [
    meta_df, metal_df, shadow_df, lovo_df, hetz_df, med_df, # Result 1
    bmi_cat_df, comorbid_df, sem_df, phewas_df,  # Result 2
    pgs_df, bmi_pgs_df, # Result 3
    ptv_protein_df, protein_bmi_df, # Result 4
    previous_meta_df, previous_meta_sex_df,
    comorbid_selection_df, # Methods
]

In [20]:
with pd.ExcelWriter('../manuscript/drafts_review1/Supplementary.xlsx', engine='xlsxwriter', mode="w") as writer:
    for i, df in enumerate(supp_dfs):
            df.to_excel(writer, sheet_name=f"Supplementary Table {i+1}", index=True)
